# Code used to create tables in the MIMIC-III paper

## Table 2: MIMIC-II patient population by critical care unit

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
%matplotlib inline

In [ ]:
# Config
sqluser = 'postgres'
dbname = 'mimic'
schema_name = 'mimiciii'

In [ ]:
# Connect to MIMIC
con = psycopg2.connect(dbname=dbname, user=sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)
# cur.close()
# con.close()

These are the details that we would like to include in Table 2, grouped by first careunit:

- Hospital admissions, no. (% of total admissions)
- Distinct ICU stays, no. (% of total unit stays)
- Age, yrs, mean ± SD
- Gender, male, percent of unit stays
- ICU length of stay, median days (IQR)
- Hospital length of stay, median days (IQR)
- ICU mortality, percent of unit stays
- Hospital mortality, percent of unit stays

### Extract and review the data

In [ ]:
# Join admissions, icustays, and patients tables

query = \
"""
SELECT a.subject_id, a.hadm_id, i.icustay_id, 
    a.admittime as hosp_admittime, a.dischtime as hosp_dischtime, 
    i.first_careunit, 
    DENSE_RANK() over(PARTITION BY a.hadm_id ORDER BY i.intime ASC) as icu_seq,
    p.dob, p.dod, i.intime as icu_intime, i.outtime as icu_outtime, 
    i.los as icu_los,
    round((EXTRACT(EPOCH FROM (a.dischtime-a.admittime))/60/60/24) :: NUMERIC, 4) as hosp_los, 
    p.gender, 
    round((EXTRACT(EPOCH FROM (a.admittime-p.dob))/60/60/24/365.242) :: NUMERIC, 4) as age_hosp_in,
    round((EXTRACT(EPOCH FROM (i.intime-p.dob))/60/60/24/365.242) :: NUMERIC, 4) as age_icu_in,
    hospital_expire_flag,
    CASE WHEN p.dod IS NOT NULL 
        AND p.dod >= i.intime - interval '6 hour'
        AND p.dod <= i.outtime + interval '6 hour' THEN 1 
        ELSE 0 END AS icu_expire_flag
FROM admissions a
INNER JOIN icustays i
ON a.hadm_id = i.hadm_id
INNER JOIN patients p
ON a.subject_id = p.subject_id
ORDER BY a.subject_id, i.intime;
"""

query_output = pd.read_sql_query(query,con)
print(query_output.head())

### Hospital admissions, no. (% of total admissions)

In [ ]:
print('\nTotal hospital admissions: {}')\
    .format(len(query_output.hadm_id.unique()))

print('\nNumber of hospital admissions by first careunit:\n')
print(query_output[['first_careunit','hadm_id']] \
                    .drop_duplicates(['hadm_id']) \
                    .groupby('first_careunit').count()).T
    
print('\nProportion of total hospital admissions:\n')
print(query_output[['first_careunit','hadm_id']] \
                    .drop_duplicates(['hadm_id']) \
                    .groupby('first_careunit') \
                    .count()/len(query_output.hadm_id.unique())*100).T

### Distinct ICU stays, no. (% of total unit stays)

In [ ]:
print('\nTotal ICU stays: {}')\
    .format(len(query_output.icustay_id.unique()))

print('\nNumber of ICU stays by careunit:\n')
print(query_output[['first_careunit','icustay_id']] \
          .groupby('first_careunit').count()).T

print('\nProportion of total ICU stays:\n')
print(query_output[['first_careunit','icustay_id']] \
          .groupby('first_careunit') \
          .count()/len(query_output.icustay_id.unique())*100).T

### Age, yrs, median ± IQR

In [ ]:
# Better to report median IQR because >89 appear as 300
print('Median age, years: {} ').format(query_output.age_icu_in.median())
print('Lower quartile age, years: {} ').format(query_output.age_icu_in.quantile(0.25))
print('Upper quartile age, years: {} \n ').format(query_output.age_icu_in.quantile(0.75))

print('Median age by careunit, years:\n ')
print(query_output[['first_careunit','age_icu_in']] \
      .groupby('first_careunit').median()).T

print('\nLower quartile by careunit, years:\n ')
print(query_output[['first_careunit','age_icu_in']] \
      .groupby('first_careunit').quantile(0.25)).T

print('\nUpper quartile by careunit, years:\n ')
print(query_output[['first_careunit','age_icu_in']] \
      .groupby('first_careunit').quantile(0.75)).T


### Gender, male, percent of unit stays

In [ ]:
print('Gender:\n')
print(query_output.loc[query_output.icu_seq==1].groupby('gender').gender.count())
print(query_output.loc[query_output.icu_seq==1].groupby('gender').gender.count() \
     /query_output.loc[query_output.icu_seq==1].gender.count()*100)

print('Gender by careunit:\n')
print(query_output.loc[query_output.icu_seq==1] \
    .groupby(['first_careunit','gender']).gender.count())

print('\nProportion by unit:\n')
print(query_output.loc[query_output.icu_seq==1] \
    .groupby(['first_careunit','gender']) \
    .gender.count()/query_output.loc[query_output.icu_seq==1] \
    .groupby(['first_careunit']).gender.count())*100

### ICU length of stay, median days (IQR)

In [ ]:
print('Median ICU length of stay, days: {}').format(query_output.icu_los.median())
print('Lower quartile ICU length of stay, days: {}') \
    .format(query_output.icu_los.quantile(0.25))
print('Upper quartile ICU length of stay, days: {}\n') \
    .format(query_output.icu_los.quantile(0.75))

print('Median length of ICU stay by careunit, days:\n ')
print(query_output[['first_careunit','icu_los']] \
      .groupby('first_careunit').median()).T

print('\nLower quartile length of ICU stay, days:\n ')
print(query_output[['first_careunit','icu_los']] \
      .groupby('first_careunit').quantile(0.25)).T

print('\nUpper quartile length of ICU stay, days:\n ')
print(query_output[['first_careunit','icu_los']] \
      .groupby('first_careunit').quantile(0.75)).T


### Hospital length of stay, median days (IQR)

In [ ]:
# NB: hadm_id is repeated in data due to multiple ICU stays
# ...so need to drop duplicates
print('Median length of hospital stay, days: {}') \
    .format(query_output.drop_duplicates(['hadm_id']).hosp_los.median())
print('Lower quartile length of hospital stay, days: {}') \
    .format(query_output.drop_duplicates(['hadm_id']).hosp_los.quantile(0.25))
print('Upper quartile length of hospital stay, days: {}\n') \
    .format(query_output.drop_duplicates(['hadm_id']).hosp_los.quantile(0.75))

print('Median length of hospital stay, days:\n ')
print(query_output.drop_duplicates(['hadm_id']) \
      [['first_careunit','hosp_los']] \
      .groupby('first_careunit').median()).T

print('\nLower quartile length of hospital stay, days:\n ')
print(query_output.drop_duplicates(['hadm_id']) \
      [['first_careunit','hosp_los']] \
      .groupby('first_careunit').quantile(0.25)).T

print('\nUpper quartile length of hospital stay, days:\n ')
print(query_output.drop_duplicates(['hadm_id']) \
      [['first_careunit','hosp_los']] \
      .groupby('first_careunit').quantile(0.75)).T

### ICU mortality, percent of unit stays

In [ ]:
print('ICU mortality, number:\n')
print(query_output \
    .groupby(['icu_expire_flag']) \
    .icu_expire_flag.count())

print('\nICU mortality, %:\n')
print(query_output.groupby(['icu_expire_flag']) \
    .icu_expire_flag.count() / query_output.icu_expire_flag.count()*100)

print('\nICU mortality by careunit:\n')
print(query_output \
    .groupby(['first_careunit','icu_expire_flag']) \
    .icu_expire_flag.count())

print('\nProportion by unit:\n')
print(query_output \
    .groupby(['first_careunit','icu_expire_flag']) \
    .icu_expire_flag.count()/query_output \
    .groupby(['first_careunit']).icu_expire_flag.count())*100

### Hospital mortality, percent of unit stays

In [ ]:
# NB: hadm_id is repeated in data due to multiple ICU stays
# ...so need to drop duplicates
print('Hospital mortality, number:\n')
print(query_output.drop_duplicates(['hadm_id']) \
    .groupby(['hospital_expire_flag']) \
    .hospital_expire_flag.count())

print('\nHospital mortality, %:\n')
print(query_output.drop_duplicates(['hadm_id']) \
    .groupby(['hospital_expire_flag']) \
    .hospital_expire_flag.count() \
    / query_output.drop_duplicates(['hadm_id']).hospital_expire_flag.count()*100)

print('\nHospital mortality:\n')
print(query_output.drop_duplicates(['hadm_id']) \
    .groupby(['first_careunit','hospital_expire_flag']) \
    .hospital_expire_flag.count())

print('\nProportion by unit:\n')
print(query_output.drop_duplicates(['hadm_id']) \
    .groupby(['first_careunit','hospital_expire_flag']) \
    .hospital_expire_flag.count()/query_output.drop_duplicates(['hadm_id']) \
    .groupby(['first_careunit']).hospital_expire_flag.count())*100

## Table 3: Distribution of primary ICD-9 codes in MIMIC-II

In [ ]:
# Connect to MIMIC
con = psycopg2.connect(dbname=dbname, user=sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)
# cur.close()
# con.close()

In [ ]:
# ICD Diagnoses are associated with hospital admissions, not ICU stay
# ...so select first ICU stay for breakdown.

query = \
"""
WITH diagnoses_icu AS (
SELECT a.subject_id, a.hadm_id, i.icustay_id, 
    a.admittime as hosp_admittime, a.dischtime as hosp_dischtime, 
    i.first_careunit, 
    DENSE_RANK() over(PARTITION BY a.hadm_id ORDER BY i.intime ASC) as icu_seq,
    p.dob, p.dod, i.intime as icu_intime, i.outtime as icu_outtime, 
    i.los as icu_los,
    round((EXTRACT(EPOCH FROM (a.dischtime-a.admittime))/60/60/24) :: NUMERIC, 4) as hosp_los, 
    p.gender, 
    round((EXTRACT(EPOCH FROM (a.admittime-p.dob))/60/60/24/365.242) :: NUMERIC, 4) as age_hosp_in,
    round((EXTRACT(EPOCH FROM (i.intime-p.dob))/60/60/24/365.242) :: NUMERIC, 4) as age_icu_in,
    hospital_expire_flag,
    CASE WHEN p.dod IS NOT NULL 
        AND p.dod >= i.intime - interval '6 hour'
        AND p.dod <= i.outtime + interval '6 hour' THEN 1 
        ELSE 0 END AS icu_expire_flag
    FROM admissions a
    INNER JOIN icustays i
    ON a.hadm_id = i.hadm_id
    INNER JOIN patients p
    ON a.subject_id = p.subject_id
    ORDER BY a.subject_id, i.intime)
SELECT d.subject_id, d.hadm_id, d.icustay_id, d.first_careunit,
    icd.seq_num, icd.icd9_code, left(icd.icd9_code,3) AS icd_first3, 
    CASE
        WHEN lower(LEFT(icd.icd9_code,1)) = 'e' THEN NULL
        WHEN lower(LEFT(icd.icd9_code,1)) = 'v' THEN NULL
        ELSE CAST( LEFT(icd.icd9_code,3) AS INT) END AS icd_first3_num,
    CASE 
        WHEN lower(LEFT(icd.icd9_code,1)) = 'e' THEN 9
        WHEN lower(LEFT(icd.icd9_code,1)) = 'v' THEN 9
        WHEN CAST( LEFT(icd.icd9_code,3) AS INT) >=0 
            AND CAST( LEFT(icd.icd9_code,3) AS INT) <= 139 THEN 0
        WHEN CAST( LEFT(icd.icd9_code,3) AS INT) >=140 
            AND CAST( LEFT(icd.icd9_code,3) AS INT) <= 239 THEN 1
        WHEN CAST( LEFT(icd.icd9_code,3) AS INT) >=240 
            AND CAST( LEFT(icd.icd9_code,3) AS INT) <= 279 THEN 2
        WHEN CAST( LEFT(icd.icd9_code,3) AS INT) >=390 
            AND CAST( LEFT(icd.icd9_code,3) AS INT) <= 459 THEN 3
        WHEN CAST( LEFT(icd.icd9_code,3) AS INT) >=460 
            AND CAST( LEFT(icd.icd9_code,3) AS INT) <= 519 THEN 4
        WHEN CAST( LEFT(icd.icd9_code,3) AS INT) >=520 
            AND CAST( LEFT(icd.icd9_code,3) AS INT) <= 579 THEN 5
        WHEN CAST( LEFT(icd.icd9_code,3) AS INT) >=580 
            AND CAST( LEFT(icd.icd9_code,3) AS INT) <= 629 THEN 6
        WHEN CAST( LEFT(icd.icd9_code,3) AS INT) >=800 
            AND CAST( LEFT(icd.icd9_code,3) AS INT) <= 959 THEN 7
        WHEN CAST( LEFT(icd.icd9_code,3) AS INT) >=960 
            AND CAST( LEFT(icd.icd9_code,3) AS INT) <= 989 THEN 8
        ELSE 9 END AS icd_first3_grp, 
    d_icd.short_title
FROM diagnoses_icu d
INNER JOIN diagnoses_icd icd
    ON d.hadm_id = icd.hadm_id
INNER JOIN d_icd_diagnoses d_icd
    ON icd.icd9_code = d_icd.icd9_code
WHERE icu_seq = 1
    AND seq_num =1;
"""

query_output = pd.read_sql_query(query,con)
print(query_output.head())

### Most common ICD-9 codes

In [ ]:
print('Primary ICD diagnoses by frequency:\n')
print(query_output['icd9_code'].value_counts().head())

print('\nPrimary ICD diagnoses, %:\n')
print(query_output['icd9_code'].value_counts().head() \
     /len(query_output['icd9_code'])*100)

### ICD-9 codes by careunit

In [ ]:
print('Primary ICD diagnoses by first ICU stay:\n')
a=query_output.loc[query_output.first_careunit=='CCU']['icd_first3_grp'].value_counts()
b=query_output.loc[query_output.first_careunit=='CSRU']['icd_first3_grp'].value_counts()
c=query_output.loc[query_output.first_careunit=='MICU']['icd_first3_grp'].value_counts()
d=query_output.loc[query_output.first_careunit=='NICU']['icd_first3_grp'].value_counts()
e=query_output.loc[query_output.first_careunit=='SICU']['icd_first3_grp'].value_counts()
f=query_output.loc[query_output.first_careunit=='TSICU']['icd_first3_grp'].value_counts()
g=query_output['icd_first3_grp'].value_counts()
df_num=pd.concat([a,b,c,d,e,f,g],axis=1)
df_num.columns = ['CCU', 'CSRU','MICU','NICU','SICU','TSICU','Total']
# Append a totals row
print(df_num.append(df_num.sum(), ignore_index=True))

print('\nProportion by careunit:\n')
a=(a/query_output.loc[query_output.first_careunit=='CCU']['icd_first3_grp'].count())*100
b=(b/query_output.loc[query_output.first_careunit=='CSRU']['icd_first3_grp'].count())*100
c=(c/query_output.loc[query_output.first_careunit=='MICU']['icd_first3_grp'].count())*100
d=(d/query_output.loc[query_output.first_careunit=='NICU']['icd_first3_grp'].count())*100
e=(e/query_output.loc[query_output.first_careunit=='SICU']['icd_first3_grp'].count())*100
f=(f/query_output.loc[query_output.first_careunit=='TSICU']['icd_first3_grp'].count())*100
g=(g/query_output['icd_first3_grp'].count())*100
df_percent=pd.concat([a,b,c,d,e,f,g],axis=1)
df_percent.columns = ['CCU', 'CSRU','MICU','NICU','SICU','TSICU','Total']
# Append a totals row
print(df_percent.append( df_num.sum(), ignore_index=True) / df_num.Total.sum() * 100)

### Distribution of ICD codes, normalised by careunit

In [ ]:
df_percent.fillna(value=0, inplace=True)
data = df_percent[['CCU', 'CSRU','MICU','NICU','SICU','TSICU']].values
row_labels = list(['CCU', 'CSRU','MICU','NICU','SICU','TSICU'])
column_labels = list(['0','1','2','3','4','5','6','7','8','9'])
fig, ax = plt.subplots()
heatmap = ax.pcolor(data, cmap=plt.cm.jet)

# move labels to top
ax.xaxis.tick_top()

# put the major ticks at the middle of each cell
ax.set_xticks(np.arange(data.shape[1])+0.5, minor=False)
ax.set_yticks(np.arange(data.shape[0])+0.5, minor=False)

ax.set_xticklabels(row_labels, minor=False)
ax.set_yticklabels(column_labels, minor=False)
plt.show()